##### Note: 
The purpose of this script is to parse the source data directories, full of .TIFs, and for each .TIF, chop it up into equally sized subimages, then take each subimage and produce 4 separate rotated images, and finally put all of them into the approprate new directory. 

In [1]:
# Necessary Imports:

from PIL import Image
import os
import matplotlib.pyplot as plt
import math

##### Setting Globals:

In [3]:
# the dimensions of the source images (5:4)
source_image_dimensions = (1280, 1024)
# 1024x1024 are the recommended largest dimensions
crop_dimensions = (256, 256)
pool = False  # set to true if you want pooling to take place
pool_dimensions = (256, 256)  # 32x32 are the recommended smallest dimensions
number_of_images_from_source = 20
dispersion_times = ['01', '03', '08', '15']
print("Dimension of new images: " +
      str(crop_dimensions[0]) + " x " + str(crop_dimensions[1]))
images_from_source_image = 4*math.floor(source_image_dimensions[0]/crop_dimensions[0])*math.floor(source_image_dimensions[1]/crop_dimensions[1])
print("Number of augmented image from 1 source image: " + str(images_from_source_image))
print("Number of augmented image from entire directory : ~" + str(4*281*images_from_source_image))

Dimension of new images: 256 x 256
Number of augmented image from 1 source image: 80
Number of augmented image from entire directory : ~89920


In [4]:
# Setting locations for input and output directories

source_data_location = '/Users/johnmays/Documents/Wirth Lab/still_data/dataset_1/'
augmented_data_location = '/Users/johnmays/Documents/Wirth Lab/still_data/Post-Hinc_small_dasets/'

In [7]:
# Create Directories:

os.chdir(augmented_data_location)
new_dir_name = "Post-Hinc_" + str(crop_dimensions[0]) + "_" + str(crop_dimensions[1])
if pool:
    new_dir_name = new_dir_name + "pooled_to" + \
        str(pool_dimensions[0]) + "_" + str(pool_dimensions[1])
os.makedirs(new_dir_name)
os.chdir(augmented_data_location + new_dir_name)
for dispersion_time in dispersion_times:
    os.makedirs(dispersion_time)

In [8]:
# Find smallest number of images in a source directory; which will be the limiting number, in order to evenly represent the dispersion times

min_number_of_images_in_subdirectory = 1000000

os.chdir(source_data_location)
for subdirectory in os.listdir(source_data_location):
    if(subdirectory != '.DS_Store'):
        number_of_images_in_subdirectory = 0
        for filename in os.listdir(source_data_location + subdirectory):
            if filename.endswith('.tif'):
                number_of_images_in_subdirectory += 1
        if number_of_images_in_subdirectory < min_number_of_images_in_subdirectory:
            min_number_of_images_in_subdirectory = number_of_images_in_subdirectory

print("Smallest number of image in a subdirectory: ", min_number_of_images_in_subdirectory)

Smallest number of image in a subdirectory:  281


In [10]:
# Creating the list of (L, T, R, B) tuples to describe the coordinates of the crops

crop_coordinates = []
top_corner = (0, 0)
bottom_corner = crop_dimensions

for x in range(0, source_image_dimensions[0], crop_dimensions[0]):
    for y in range(0, source_image_dimensions[1], crop_dimensions[1]):
        corner_coordinates = (
            x, y) + (x+crop_dimensions[0], y+crop_dimensions[1])
        crop_coordinates.append(corner_coordinates)

print(crop_coordinates)

[(0, 0, 256, 256), (0, 256, 256, 512), (0, 512, 256, 768), (0, 768, 256, 1024), (256, 0, 512, 256), (256, 256, 512, 512), (256, 512, 512, 768), (256, 768, 512, 1024), (512, 0, 768, 256), (512, 256, 768, 512), (512, 512, 768, 768), (512, 768, 768, 1024), (768, 0, 1024, 256), (768, 256, 1024, 512), (768, 512, 1024, 768), (768, 768, 1024, 1024), (1024, 0, 1280, 256), (1024, 256, 1280, 512), (1024, 512, 1280, 768), (1024, 768, 1280, 1024)]


In [ ]:
# cropping, rotating, and depositing images:

for dispersion_time in dispersion_times:
    destination_dir = augmented_data_location + new_dir_name + "/" + dispersion_time
    i = 0
    for filename in os.listdir(source_data_location + dispersion_time):
        i += 1
        if i > min_number_of_images_in_subdirectory:
            break
        elif filename.endswith('.tif'):
            os.chdir(source_data_location + dispersion_time)
            img = Image.open(filename)
            os.chdir(destination_dir)
            for i in range(len(crop_coordinates)):
                # L, T, R, B (does not include pixel on bottom-right coordinate)
                cropped_img = img.crop(crop_coordinates[i])
                if not pool:
                    cropped_img.save(
                        filename[0:-4] + '_' + str(crop_dimensions[0]) + '_cropped_0_' + str(i) + '.tif')
                    cropped_img.rotate(90).save(
                        filename[0:-4] + '_' + str(crop_dimensions[0]) + '_cropped_90' + str(i) + '.tif')
                    cropped_img.rotate(180).save(
                        filename[0:-4] + '_' + str(crop_dimensions[0]) + '_cropped_180' + str(i) + '.tif')
                    cropped_img.rotate(270).save(
                        filename[0:-4] + '_' + str(crop_dimensions[0]) + '_cropped_270' + str(i) + '.tif')
                else:
                    resized_img = cropped_img.resize(pool_dimensions)
                    resized_img.save(
                        filename[0:-4] + '_' + str(pool_dimensions[0]) + '_cropped_pooled_0' + str(i) + '.tif')
                    resized_img.rotate(90).save(
                        filename[0:-4] + '_' + str(pool_dimensions[0]) + '_cropped_pooled_90' + str(i) + '.tif')
                    resized_img.rotate(180).save(
                        filename[0:-4] + '_' + str(pool_dimensions[0]) + '_cropped_pooled_180' + str(i) + '.tif')
                    resized_img.rotate(270).save(
                        filename[0:-4] + '_' + str(pool_dimensions[0]) + '_cropped_pooled_270' + str(i) + '.tif')
        else:
            continue